In [1]:
import gdown
import pandas as pd

In [2]:
# Import des bibliothèques de viz
import matplotlib.pyplot as plt
import seaborn as sns

# Import split data
from sklearn.model_selection import train_test_split

# Import modèle de ML NON Supervisé
from sklearn.neighbors import NearestNeighbors

# Import outil standardisation de la donnée
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

# Import pipeline
from sklearn.pipeline import Pipeline

# Gestion des warnings
import warnings

In [3]:
# import du csv data_ml
file = "1h6zW1eKmDVZCJ2d2prtjVV0HqxlPMGP9"
gdown.download(f"https://drive.google.com/uc?id={file}", "data_ml.csv", quiet=False)
df = pd.read_csv("data_ml.csv")

Downloading...
From (original): https://drive.google.com/uc?id=1h6zW1eKmDVZCJ2d2prtjVV0HqxlPMGP9
From (redirected): https://drive.google.com/uc?id=1h6zW1eKmDVZCJ2d2prtjVV0HqxlPMGP9&confirm=t&uuid=8aea9489-0153-4cfe-af64-dea612df08c7
To: C:\Users\kwind\PROJET2\data_ml.csv
100%|███████████████████████████████████████████████████████████████████████████████| 176M/176M [00:04<00:00, 38.7MB/s]


In [4]:
# supprimer les films avec valeurs nulles dans Date de création et genres
# ne garder que les films avec minimum 500 votes et une note de plus de 1 (exclut les Nan en même temps) 
df = df.dropna(subset=['Date de création','genres'])
df_noted = df[(df['Note moyenne'] > 1.0) & (df['numVotes']>500)].copy()

In [5]:
df_noted['Date de création'] = df_noted['Date de création'].astype(int)

In [6]:
df_noted['Titre originale LC'] = df_noted['Titre originale'].str.lower()

In [14]:
df_noted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67773 entries, 1 to 732628
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   t_id                67773 non-null  object 
 1   primaryTitle        67773 non-null  object 
 2   Titre originale     67773 non-null  object 
 3   Date de création    67773 non-null  int64  
 4   Durée (Min)         67544 non-null  float64
 5   genres              67773 non-null  object 
 6   directeur_id        67740 non-null  object 
 7   Note moyenne        67773 non-null  float64
 8   numVotes            67773 non-null  float64
 9   titre_FR            39969 non-null  object 
 10  langue_originale    62696 non-null  object 
 11  chemin_affiche      61996 non-null  object 
 12  description         62032 non-null  object 
 13  directeur_name      67740 non-null  object 
 14  Action              67773 non-null  int64  
 15  Adventure           67773 non-null  int64  
 16  Animatio

In [7]:
# créer un nouvelle colonne note_ponderee qui permettra de trier les voisins
C = df_noted['Note moyenne'].mean()
m = df_noted['numVotes'].quantile(0.5) # pour les films en-dessous de m (environ 1800), la note sera tiré vers la moyenne

df_noted['Note_ponderee'] = (
    (df_noted['numVotes'] / (df_noted['numVotes'] + m)) * df_noted['Note moyenne']
    + (m / (df_noted['numVotes'] + m)) * C
)

In [8]:
# features
X = df_noted[['Action','Adventure','Animation','Biography','Comedy','Crime',
                'Documentary','Drama','Family','Fantasy','Film-Noir', 'History','Horror',
                'Music','Musical','Mystery','News', 'Romance','Sci-Fi','Short','Sport',
                'Thriller','War','Western','Date de création']]
num_features = ['Date de création']
bin_features = ['Action','Adventure','Animation','Biography','Comedy','Crime',
                'Documentary','Drama','Family','Fantasy','Film-Noir', 'History','Horror',
                'Music','Musical','Mystery','News', 'Romance','Sci-Fi','Short','Sport',
                'Thriller','War','Western']

In [9]:
# standardiser les colonnes 

preprocessor = ColumnTransformer(
    transformers=[
        ('num',MinMaxScaler(), num_features),
        ('bin','passthrough', bin_features)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', NearestNeighbors(metric='euclidean'))
])



In [10]:
# créer le modèle 
nn_model = NearestNeighbors(metric='euclidean')

# entraîner sur les données standardisées X_scaled
pipeline.fit(X) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['Date de création']),
                                                 ('bin', 'passthrough',
                                                  ['Action', 'Adventure',
                                                   'Animation', 'Biography',
                                                   'Comedy', 'Crime',
                                                   'Documentary', 'Drama',
                                                   'Family', 'Fantasy',
                                                   'Film-Noir', 'History',
                                                   'Horror', 'Music', 'Musical',
                                                   'Mystery', 'News', 'Romance',
                                                   'Sci-Fi', 'Short', 'Sport',
                                                   'Thriller', 'War',
                                                   'Western'])])),
                ('model', NearestNeighbors(metric='euclidean'))])

In [11]:
print(X.index.equals(df_noted.index))

True


In [26]:
def recommander_film(titre, n=10):

    # passer le titre cible en minuscule
    titre = titre.lower()
    
    # trouver le film dans le df_noted
    film_row = df_noted[df_noted['Titre originale LC'] == titre]

    if film_row.empty:
        return f"Le film '{titre}' n'a pas été trouvé dans le jeu de données d'entraînement."

    # utiliser l'index du premier résultat trouvé (peut-etre changer en dernier ?
    idx_label = film_row.index[0]
    
    # extraire le vecteur de features NON SCALÉ du film cible, en utilisant .loc (label index)
    film_df_unscaled = X.loc[[idx_label]]
    
    # Trouver 50 voisins sur le DataFrame de features non scalées (film_df_unscaled)
    distances, indices = pipeline.named_steps['model'].kneighbors(
        pipeline.named_steps['preprocessor'].transform(film_df_unscaled), # On transforme la donnée AVANT de l'envoyer à kneighbors
        n_neighbors=50
    )

    # Les indices renvoyés par nn_model sont des indices POSITIONNELS dans X_scaled/df_noted
    # utiliser .iloc sur df_noted pour récupérer les lignes correspondantes
    indices = indices[0]
    
    # récupérer les données complètes des films trouvés
    recos = df_noted.iloc[indices][['Titre originale', 'genres', 'Date de création', 'Note moyenne',"numVotes", 'Note_ponderee', 'directeur_name','Titre originale LC']]

    # exclure explicitement le film cible par titre ou index
    recos = recos[recos.index != idx_label]
    recos = recos[recos['Titre originale LC'] != titre]
    
    # trier les recommandations par Note pondérée décroissante
    recos_triées = recos.sort_values(by='Note_ponderee', ascending=False)

    # retourner le top 10
    return recos_triées[['Titre originale', 'genres', 'Date de création', 'Note moyenne','directeur_name']].head(n)

In [25]:
recommander_film("Pulp Fiction", n=10)

,Titre originale,genres,Date de création,Note moyenne,numVotes,directeur_name
78024,Casino,"Crime,Drama",1995,8.2,607521.0,Martin Scorsese
80907,Trainspotting,"Crime,Drama",1996,8.1,760549.0,Danny Boyle
78377,La haine,"Crime,Drama",1995,8.1,219520.0,Mathieu Kassovitz
79904,Eskiya,"Crime,Drama",1996,8.1,74704.0,Yavuz Turgul
92459,Mahanadhi,"Crime,Drama",1994,8.6,5008.0,Santhana Bharathi
77092,Once Were Warriors,"Crime,Drama",1994,7.9,38811.0,Lee Tamahori
74568,A Bronx Tale,"Crime,Drama",1993,7.8,179050.0,Robert De Niro
78125,Dead Man Walking,"Crime,Drama",1995,7.5,106973.0,Tim Robbins
74682,Damini,"Crime,Drama",1993,7.8,4307.0,Rajkumar Santoshi
75821,Udzinarta mze,"Crime,Drama",1992,8.4,1734.0,Temur Babluani


In [19]:
df_noted[df_noted['t_id'] == "77201"][['Titre originale', 'Date de création', 'genres', 'Note moyenne']]

,Titre originale,Date de création,genres,Note moyenne
